In [21]:
import cv2 as cv
import numpy as np
import os
import shutil
import random
import math
import shutil
import asyncio
import json


def rotate_image(image, angle):
    height, width = image.shape[:2]
    ##
    diagonal = int(math.ceil(math.sqrt(height**2 + width**2)))

    delta_w = diagonal - width
    delta_h = diagonal - height
    top = delta_h // 2
    bottom = delta_h - top
    left = delta_w // 2
    right = delta_w - left

    image = cv.copyMakeBorder(
        image, top, bottom, left, right, cv.BORDER_CONSTANT, value=(0, 0, 0))

    image_center = diagonal // 2, diagonal // 2
    print(image_center)
    rotation_mat = cv.getRotationMatrix2D(image_center, angle, 1)

    rotated = cv.warpAffine(image, rotation_mat,
                            (diagonal, diagonal), borderValue=(0, 0, 0, 0))

    return rotated


def trim(img):
    # get the center of the image
    height, width = img.shape[:2]
    center_x = width // 2
    center_y = height // 2
    cv.imwrite('test.png', img)
    # get top bound
    for i in range(center_y):
        if np.any(img[i, :] != 0):
            top = i
            break
    # get bottom bound
    for i in range(height - 1, center_y, -1):
        if np.any(img[i, :] != 0):
            bottom = i
            break
    # get left bound
    for i in range(center_x):
        if np.any(img[:, i] != 0):
            left = i
            break
    # get right bound
    for i in range(width - 1, center_x, -1):
        if np.any(img[:, i] != 0):
            right = i
            break
    return img[top:bottom, left:right]


def rotate_and_trim(img, angle):
    return trim(rotate_image(img, angle))

def anti_rotate(path):
    raster_path = f'{path}/raster'
    anti_rotate_path = f'{path}/anti_rotate'
    os.makedirs(anti_rotate_path, exist_ok=True)
    ## open json
    with open(f'{path}/positions.json', 'r') as f:
        positions = json.load(f)['positions']
    rasters = os.listdir(raster_path)
    ## sort
    rasters.sort(key=lambda x: int(x.split('.')[0]))
    for i, file in enumerate(rasters):
        if file.endswith('.png'):
            img = cv.imread(f'{raster_path}/{file}', cv.IMREAD_UNCHANGED)
            rotation = positions[i*3+2]
            print(f'{anti_rotate_path}/{file}')

            img = rotate_and_trim(img, -rotation)
            cv.imwrite(f'{anti_rotate_path}/{file}', img)

anti_rotate('puzzle/0-C')






puzzle/0-C/anti_rotate/0.png
(494, 494)
puzzle/0-C/anti_rotate/1.png
(542, 542)
puzzle/0-C/anti_rotate/2.png
(589, 589)
puzzle/0-C/anti_rotate/3.png
(584, 584)
puzzle/0-C/anti_rotate/4.png
(563, 563)
puzzle/0-C/anti_rotate/5.png
(623, 623)
puzzle/0-C/anti_rotate/6.png
(540, 540)
puzzle/0-C/anti_rotate/7.png
(518, 518)
puzzle/0-C/anti_rotate/8.png
(586, 586)
puzzle/0-C/anti_rotate/9.png
(590, 590)
puzzle/0-C/anti_rotate/10.png
(579, 579)
puzzle/0-C/anti_rotate/11.png
(631, 631)
puzzle/0-C/anti_rotate/12.png
(542, 542)
puzzle/0-C/anti_rotate/13.png
(590, 590)
puzzle/0-C/anti_rotate/14.png
(590, 590)
puzzle/0-C/anti_rotate/15.png
(589, 589)
puzzle/0-C/anti_rotate/16.png
(588, 588)
puzzle/0-C/anti_rotate/17.png
(582, 582)
puzzle/0-C/anti_rotate/18.png
(585, 585)
puzzle/0-C/anti_rotate/19.png
(574, 574)
puzzle/0-C/anti_rotate/20.png
(583, 583)
puzzle/0-C/anti_rotate/21.png
(422, 422)
puzzle/0-C/anti_rotate/22.png
(523, 523)
puzzle/0-C/anti_rotate/23.png
(586, 586)
puzzle/0-C/anti_rotate/24.

In [28]:
import os
import json
import cv2
path = 'puzzle/0-C'
anti_rotate_path = f'{path}/anti-rotate'
positions_path = f'{path}/positions.json'
positions_file = open(positions_path, 'r')
positions = json.load(positions_file)['positions']
positions_file.close()

## sort raster files
anti_rotate_files = os.listdir(anti_rotate_path)
anti_rotate_files.sort(key=lambda x: int(x.split('.')[0]))

img_elements = []
for i, anti_rotate_file in enumerate(anti_rotate_files):
    height,width = cv2.imread(f'{anti_rotate_path}/{anti_rotate_file}').shape[:2]
    left, top, rotation = positions[3*i+0], positions[3*i+1], positions[3*i+2]
    element = f'''
    <div id='p-{i}' class='p pc-{i}' style='left:{left}px;top:{top}px;'>
        <img class='p-img' src='anti-rotate/{i}.png?1688235665' width='{width}' height='{height}'>
    </div>'''
    img_elements.append(element)


html = f'''
<!doctype html>
<html>
<head>
<title>Cut Proof - 100</title>
<link rel="stylesheet" media="all" href="sprite_p.css">
<style>

body {{
background: black;
color: white;
}}
.container  {{
position: relative;
display: flex;
flex-wrap: wrap;
}}
.p  {{
transition: opacity linear 0.5s;
background-image: none;
}}
input[name=assembled]:checked + .container .p  {{
position: absolute;
}}
.p.is-highlight,
.p:hover,
.p:active  {{
opacity: 0;
}}


</style>
</head>
<body>
<p>
Piece count: 48<br>
<button>
<label for="assembled">Toggle Assembled State</label>
</button>
</p>

<!-- All the piece div elements -->
<input type="checkbox" checked id="assembled" name="assembled">
<div class="container">
{
    ''.join(img_elements)
}
</div>

<script>
/* Toggle the adjacent pieces next to the one that is clicked. */
fetch("/adjacent.json")
.then(response => response.json())
.then(adjacent => {{
    document.addEventListener('click', (event) => {{
        for (let target = event.target; target && target != this; target = target.parentNode) {{
            if (target.classList && target.classList.contains('p')) {{
                const $piece = target;
                const piece_id = $piece.getAttribute("id").replace("p-", "");
                const adjacent_piece_ids = adjacent[piece_id];
                adjacent_piece_ids
                    .map(pc => {{return document.getElementById("p-"+pc)}})
                    .map(el => el.classList.toggle('is-highlight'))
                break;
            }}
        }}
    }}, false);
}});
</script>

</body>
</html>'''

with open(f'{path}/test.html', 'w') as f:
    f.write(html)